In [1]:
# Predict the model, get the model from mlflow registery and predict.

import mlflow
import dagshub
import pandas as pd

# Initialize the dagshub repo information.
dagshub.init(repo_owner='pavipd495', repo_name='spam_or_ham_fsec', mlflow=True)


# Model to pull.
model_name = "Bidirectional_lstm"
model_version = 1
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")


Accessing as pavipd495

Initialized MLflow to track repo "pavipd495/spam_or_ham_fsec"

Repository pavipd495/spam_or_ham_fsec initialized!

2024-12-02 22:33:41.989331: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 22:33:42.102811: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 22:33:42.198365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733159022.280452  183072 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733159022.333660  183072 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 22:33:42.553170: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [7]:
# predict.
test_data = pd.read_csv("test_data.csv")
y_pred = model.predict(test_data)


y_pred_binary = (y_pred >= 0.5).astype(int)

# The prediction will be in 0 and 1. Change it back to original values. 0 to 1 and 1 to -1.
y_pred_binary.replace(1, -1, inplace=True)
y_pred_binary.replace(0, 1, inplace=True)

40/40 ━━━━━━━━━━━━━━━━━━━━ 162s 4s/step


In [18]:
# Save the predictions into a file.
y_pred_binary.to_csv("test_labels.csv", index=False)